## forecast-then-optimize
### forecast

In [ ]:
import sys
from predictor.data import Dataset
from simulation.forecast import Forecast
from predictor.moirai2_predictor import MoiraiQuantilePredictor
from gluonts.model.predictor import Predictor
from pathlib import Path
import sys
import os

BASE_DIR="../outputs/decision"
os.makedirs(BASE_DIR, exist_ok=True)

for ds_name in ["borg2019_e","borg2019_d","huawei2025","azure2019"]:
    for test_split_offset in range(288,313):
        ds = Dataset(dataset_name=ds_name, freq='1H', test_split_offset=test_split_offset, prediction_length=24)
        predictor = MoiraiQuantilePredictor(
            model_path=f"Salesforce/moirai-2.0-R-small",
            prediction_length=ds.prediction_length,
            context_length=120,
            target_dim=1,
            past_feat_dynamic_real_dim=ds.meta_info.past_feat_dynamic_real_dim,
            batch_size=512,
            device="cuda",
        )
        forecast_obj=Forecast().predict(
            test_data=ds.test_data,
            metadata=ds.meta_info,
            model="Moirai-2.0-R-small",                  ###
            predictor=predictor,
            context_length= 120,
        )
        forecast_obj.save(BASE_DIR+'/moirai2')


### optimize

In [ ]:
from simulation.assignment_simpy import SimPyAssignment
import pandas as pd
test_split_offset=288
base_dir=BASE_DIR+"/moirai2/borg2019_e"
assignment_obj=SimPyAssignment(num_hosts=30,host_capacity=1,threshold_rate=1,scheduler_type="FFD",forecast_path=f"{base_dir}/{test_split_offset}",quantile=0.5)
assignment_obj.schedule()
cpu_util=assignment_obj.compute_util()
slav=assignment_obj.compute_SLAV()
print(cpu_util)
print(slav)
active_num=0
for host in assignment_obj.hosts.values():
    if  not all(util == 0 for util in host.real_history.values()):
        active_num+=1
print(active_num)
print((assignment_obj.mapping))